In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
import HTMLParser as htm
import string
import re

# SK-learn library for splitting data
from sklearn.model_selection import train_test_split

## Read in data

In [4]:
data = pd.read_csv("tweet_data_1.csv",sep='\t',quoting=3)
data["escape"] = data.apply(lambda row: htm.HTMLParser().unescape(row[1].decode("utf-8")),axis=1)

data.head()

,Id,Tweet,Emotion,Positive,escape
0,138881940341260288:,I got a surprise for all you bitches...pull th...,:: surprise,0,I got a surprise for all you bitches...pull th...
1,144479819843911683:,If I was a thief.. The first thing I would ste...,:: joy,1,If I was a thief.. The first thing I would ste...
2,139110849120972800:,"""&quot;@RevRunWisdom: not afraid of tomorrow, ...",:: fear,0,"""""@RevRunWisdom: not afraid of tomorrow, for I..."
3,141532076791971840:,"""Extreme can neither fight nor fly.&#xA;-- Wil...",:: fear,0,"""Extreme can neither fight nor fly.\n-- Willia..."
4,145353048817012736:,Thinks that @melbahughes had a great 50th birt...,:: surprise,0,Thinks that @melbahughes had a great 50th birt...


## Split into train & test sets

In [5]:
# Train and test data frames
train, test = train_test_split(data, test_size = 0.2)

# Train and test target labels
train_pol_y = train.ix[:,3].tolist()
test_pol_y = test.ix[:,3].tolist()

# MC NEW CODE
# Train and test x
train_pol_x = train.ix[:, 4].tolist()
test_pol_x = test.ix[:, 4].tolist()

## Preprocess & clean data

In [12]:
def process_data(data):
    """Converts to lowercase, strips out punctuation,
    removes excess whitespace within a string & leading & trailing whitespace"""
    new_list = []
    table = string.maketrans("","")
    for elem in data:
        elem = "".join(i for i in elem if ord(i)<128)
        elem = str(elem)        
        elem = elem.lower()
        elem = elem.translate(table, string.punctuation)
        elem = re.sub(' +',' ', elem)
        elem = elem.strip()
        
        new_list.append(elem)
    return new_list

train_pol_x = process_data(train_pol_x)
test_pol_x = process_data(test_pol_x)

## Pull in GLOVE embeddings

In [6]:
# Pull in word list & vectors
wordsList = np.load('wordsList.npy')
print('Loaded the word list!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')
print ('Loaded the word vectors!')

Loaded the word list!
Loaded the word vectors!


## Get Matrix IDs for Training

# CHANGE numDimensions & re-run

In [14]:
# For train_pol_x - Took approximately 25 minutes to run on 8 core compute instance

# Find tweet with most number of words
maxSeqLength = max([len(elem.split()) for elem in data.ix[:, 4]]) #Maximum number of words in a tweet
numDimensions = 10 #Dimensions for each word vector
numFiles = len(train_pol_x)

ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0

for tweet in train_pol_x:
    indexCounter = 0
    split = tweet.split()
    for word in split:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1 


# Pass into embedding function and see if it evaluates. 

np.save('idsMatrix_train_x', ids)

## Get Matrix IDs for Test

# CHANGE numDimensions & re-reun

In [16]:
# For test_pol_x - took XX minutes to run

# Find tweet with most number of words
maxSeqLength = max([len(elem.split()) for elem in data.ix[:, 4]]) #Maximum number of words in a tweet
numDimensions = 10 #Dimensions for each word vector
numFiles = len(test_pol_x)

ids = np.zeros((numFiles, maxSeqLength), dtype='int32')
fileCounter = 0

for tweet in test_pol_x:
    indexCounter = 0
    split = tweet.split()
    for word in split:
        try:
            ids[fileCounter][indexCounter] = wordsList.index(word)
        except ValueError:
            ids[fileCounter][indexCounter] = 399999 #Vector for unkown words
        indexCounter = indexCounter + 1
        if indexCounter >= maxSeqLength:
            break
    fileCounter = fileCounter + 1 


# Pass into embedding function and see if it evaluates. 

np.save('idsMatrix_test_x', ids)

In [17]:
train_ids = np.load('idsMatrix_train_x.npy')
test_ids = np.load('idsMatrix_test_x.npy')

## Helper functions for training

In [18]:
from random import randint

def getTrainBatch(train_data, train_labels, train_ids):
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    # iterate through batch size
    for i in range(batchSize):
        num = randint(1, (len(train_data)-1))

        if train_labels[num] == 1:
            labels.append([1,0])
        else:
            labels.append([0,1])
            
        arr[i] = train_ids[num-1:num]
        
    return arr, labels


def getTestBatch(test_data, test_labels, test_ids):
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    for i in range(batchSize):
        num = randint(1,(len(test_data)-1))
        
        if test_labels[num] == 1:
            labels.append([1,0])
        else:
            labels.append([0,1])
            
        arr[i] = test_ids[num-1:num]
        
    return arr, labels

## RNN Model

# CHANGE batchSize, lstmUnits & iterations & RERUN

In [34]:
# Specify parameters
batchSize = 8
lstmUnits = 4
numClasses = 2
iterations = 10000

# Reset graph & create placeholders
tf.reset_default_graph()
labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

# Lookup word vectors
data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

# Feed RNN cell
lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.75)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

# Get final output
weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

# Define correct predictions and accuracy
correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

# Define loss & optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)

## For Tensorboard

In [35]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

## For Training

In [38]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

for i in range(iterations):
    # Next Batch of reviews
    nextBatch, nextBatchLabels = getTrainBatch(train_pol_x, train_pol_y, train_ids);
    sess.run(optimizer, {input_data: nextBatch, labels: nextBatchLabels})
   
    # Write summary to Tensorboard
    if (i % 50 == 0):
        summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels})
        writer.add_summary(summary, i)

    # Save the network every 10,000 training iterations
    if (i % 10000 == 0 and i != 0):
        save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
        print("saved to %s" % save_path)
writer.close()

In [39]:
# This is to load a pre-trained model - not working currently

# sess = tf.InteractiveSession()
# saver = tf.train.Saver()
# saver.restore(sess, tf.train.latest_checkpoint('models'))

In [40]:
iterations = 10
for i in range(iterations):
    nextBatch, nextBatchLabels = getTestBatch(test_pol_x, test_pol_y, test_ids);
    print("Accuracy for this batch:", (sess.run(accuracy, {input_data: nextBatch, labels: nextBatchLabels})) * 100)

('Accuracy for this batch:', 62.5)
('Accuracy for this batch:', 62.5)
('Accuracy for this batch:', 62.5)
('Accuracy for this batch:', 62.5)
('Accuracy for this batch:', 50.0)
('Accuracy for this batch:', 62.5)
('Accuracy for this batch:', 87.5)
('Accuracy for this batch:', 62.5)
('Accuracy for this batch:', 75.0)
('Accuracy for this batch:', 87.5)
